In [3]:
from stable_baselines3 import A2C
import torch as th

model = A2C("MlpPolicy", "CartPole-v1").learn(10000)

In [2]:
import gym

from stable_baselines3 import A2C

env = gym.make("CartPole-v1")

model = A2C("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10_000)

vec_env = model.get_env()
obs = vec_env.reset()
for i in range(1000):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)
    vec_env.render()
    # VecEnv resets automatically
    # if done:
    #   obs = vec_env.reset()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 27.3     |
|    ep_rew_mean        | 27.3     |
| time/                 |          |
|    fps                | 858      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.657   |
|    explained_variance | 0.186    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -2.42    |
|    value_loss         | 18.4     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 36.5     |
|    ep_rew_mean        | 36.5     |
| time/                 |          |
|    fps                | 891      |
|    iterations         | 200      |
|    time_elapsed 

: 

In [ ]:
import stable_baselines3
from stable_baselines3.dqn import DQN
from gym import spaces

# Define the environment with five dimensions and three actions (buy, hold, and sell)
class StockTradingEnv(gym.Env):
    def __init__(self):
        super(StockTradingEnv, self).__init__()
        self.observation_space = spaces.Box(low=0, high=1, shape=(5,))
        self.action_space = spaces.Discrete(3)

    def _step(self, action):
        # Perform the selected action and observe the reward and next state
        reward = 0
        next_state = None
        done = False

        # Update the environment and return the reward, next state, and done flag
        return next_state, reward, done

# Create the stock trading environment
env = StockTradingEnv()

# Create the DQN reinforcement learning model
model = DQN(env, network='mlp')

# Train the model using the DQN reinforcement learning algorithm
model.learn(total_timesteps=1000, progress_bar=True)

# Use the trained model to make predictions on new stock data
predictions = model.predict(new_data)

In [ ]:
observation = obs.reshape((-1,) + model.observation_space.shape)
observation = obs_as_tensor(observation, device)
with th.no_grad():
    q_values = model.q_net(observation)

In [2]:
import torch as th
from stable_baselines3 import DQN

model = DQN("MlpPolicy", "CartPole-v1")
env = model.get_env()

obs = env.reset() #0번째 관측값
with th.no_grad():
     obs_tensor, _ = model.q_net.obs_to_tensor(obs)
     q_values = model.q_net(obs_tensor)

In [7]:
env

In [ ]:
# Imports
from stable_baselines3.dqn.policies import QNetwork, DQNPolicy

# Override some methods of the class QNetwork used by the DQN model in order to set to a negative value the q-values of
# some actions

# Two possibile methods to override:
# Override _predict ---> alter q-values only during predictions but not during training
# Override forward ---> alter q-values also during training (Attention: here we are working with batches of q-values)

class QNetwork_modified(QNetwork):
    
    def forward(self, obs: th.Tensor) -> th.Tensor:
        """
        Predict the q-values.
        :param obs: Observation
        :return: The estimated Q-Value for each action.
        """
        # Compute the q-values using the QNetwork
        q_values = self.q_net(self.extract_features(obs))
        # For each observation in the training batch:
        for i in range(obs.shape[0]):
            # Here you can alter q_values[i]

        
        return q_values

    
# Override the make_q_net method of the DQN policy used by the DQN model to make it use the new DQN network

class DQNPolicy_modified(DQNPolicy):
    def make_q_net(self) -> DQNPolicy:
        # Make sure we always have separate networks for features extractors etc
        net_args = self._update_features_extractor(self.net_args, features_extractor=None)
        return QNetwork_modified(**net_args).to(self.device)



model = DQN(DQNPolicy_modified, env, verbose=1)

In [ ]:
state = [0.1, 0.2, 0.3]
q_values = model.predict(state)

In [ ]:
import gym

from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy


# Create environment
env = gym.make("LunarLander-v2")

# Instantiate the agent
model = DQN("MlpPolicy", env, verbose=1)
# Train the agent and display a progress bar
model.learn(total_timesteps=int(2e5), progress_bar=True)
# Save the agent
model.save("dqn_lunar")
del model  # delete trained model to demonstrate loading

# Load the trained agent
# NOTE: if you have loading issue, you can pass `print_system_info=True`
# to compare the system on which the model was trained vs the current one
# model = DQN.load("dqn_lunar", env=env, print_system_info=True)
model = DQN.load("dqn_lunar", env=env)

# Evaluate the agent
# NOTE: If you use wrappers with your environment that modify rewards,
#       this will be reflected here. To evaluate with original rewards,
#       wrap environment in a "Monitor" wrapper before other wrappers.
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)

# Enjoy trained agent
vec_env = model.get_env()
obs = vec_env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render()